In [1103]:
import re, sys, nltk
from nltk.tokenize.stanford import StanfordTokenizer
path_to_jar = "/Users/xinsun/Desktop/Relation-Classification/stanford/stanford-postagger-2017-06-09/stanford-postagger.jar"
tokenizer = StanfordTokenizer(path_to_jar)
import _pickle as pickle

import os
from nltk.parse import stanford

# Dependency Tree
from nltk.parse.stanford import StanfordDependencyParser

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPParser instead.'
  after removing the cwd from sys.path.


### Data pre-processing to get the training data and relation label

In [1104]:
with open('data/relation-3-train.txt', encoding="latin") as f:
    file = f.read()
    print(file)

001 "The <e1>island</e1> was farmed for <e2>oats</e2> and barley until 1796 when the owner mandated that oak and alder trees be planted."
WordNet(e1) = "island%1:17:00::", WordNet(e2) = "oat%1:13:00::", Product-Producer(e2, e1) = "true", Query = "the * * farmed for *"

002 "In some Asian countries, including China, Malaysia and the Philippines, the <e1>turtles</e1> are farmed for <e2>food</e2>."
WordNet(e1) = "turtle%1:05:00::", WordNet(e2) = "food%1:03:00::", Product-Producer(e2, e1) = "false", Query = "the * * farmed for *"
Comment: violates (iii)

003 "Surrounding each neuron there was a group of cells of the glial type (satellite cells), which together with the <e1>collagen fibres</e1> composed the <e2>glial capsule</e2>."
WordNet(e1) = "fibre%1:08:00::", WordNet(e2) = "capsule%1:08:00::", Product-Producer(e2, e1) = "false", Query = "the * composed the *"
Comment: Part-Whole

004 "The soft brown and beige <e1>tones</e1> in the decoration provide a <e2>tranquil mood</e2>."
WordNet(e

In [1105]:
# Extracting the Relations 
# For training data open "TRAIN_FILE.TXT" and for test data open "TEST_FILE_FULL.TXT"

lines, rels = [], []
for line in open("data/relation-3-train.txt",  encoding="latin"):
    lines.append(line.strip())
        
d=0        
for i, w in enumerate(lines):
    m = re.match(r'^([0-9]+)\s"(.+)"$', w.strip())
    if (m is not None):
        #print(m.group(0))
        indx = lines.index(m.group(0))
        #print(d)
        d+=1
        #print(str(lines[indx+1].split(',')[2].strip()) + ',' + str(lines[indx+1].split(',')[3].strip()))
        rels.append(str(lines[indx+1].split(',')[2].strip()) + ',' + str(lines[indx+1].split(',')[3].strip()))

        
relations = []
for rel in rels:
    if rel == 'Product-Producer(e1,e2) = "true"':
        relations.append('Product-Producer(e1,e2)')  
    elif rel == 'Product-Producer(e2,e1) = "true"':
        relations.append('Product-Producer(e2,e1)')    
    else:
        relations.append('others')    

f = open("data/relation-3/relation_3_train.txt", 'w')
for rel in relations:
    print(rel)
    f.write(rel+'\n')

Product-Producer(e2,e1)
others
others
others
others
others
Product-Producer(e2,e1)
Product-Producer(e2,e1)
Product-Producer(e2,e1)
Product-Producer(e2,e1)
Product-Producer(e2,e1)
others
others
others
others
Product-Producer(e1,e2)
Product-Producer(e2,e1)
others
Product-Producer(e2,e1)
Product-Producer(e1,e2)
others
others
others
Product-Producer(e1,e2)
Product-Producer(e2,e1)
Product-Producer(e2,e1)
others
others
Product-Producer(e1,e2)
others
Product-Producer(e1,e2)
Product-Producer(e2,e1)
Product-Producer(e1,e2)
Product-Producer(e2,e1)
others
Product-Producer(e1,e2)
Product-Producer(e2,e1)
others
others
others
others
Product-Producer(e1,e2)
Product-Producer(e1,e2)
Product-Producer(e2,e1)
Product-Producer(e1,e2)
Product-Producer(e1,e2)
others
others
Product-Producer(e2,e1)
Product-Producer(e1,e2)
Product-Producer(e2,e1)
Product-Producer(e2,e1)
Product-Producer(e2,e1)
Product-Producer(e1,e2)
Product-Producer(e1,e2)
Product-Producer(e2,e1)
Product-Producer(e1,e2)
Product-Producer(e1,e2)

### Getting entities and sentences

In [1106]:
sentences = []
e1 = []
e2 = []

num = 0
for i,w in enumerate(lines):
    m = re.match(r'^([0-9]+)\s"(.+)"$', w.strip())
    
    if(m is not None):
        print('num: ', num)
        num+=1
        temporary = m.group(2)
        print(temporary)
    
    
        text = []
        temp = []
        t = temporary.split("<e1>")
        text.append(t[0])
        #print('text: ', text)

        temp.append(t[0])

        t = t[1].split("</e1>")
        e1_text = text
        e1_text = " ".join(e1_text)
        e1_text = tokenizer.tokenize(e1_text)
        #print('e1_text: ', e1_text)

        text.append(t[0])
        #print('text: ', text)

        e11= t[0]
        #print('e11: ', e11)


        y = tokenizer.tokenize(t[0])
        #print(y)
        y[0] +="E11"

        temp.append(" ".join(y))
        t = t[1].split("<e2>")
        #print(t)
        text.append(t[0])
        temp.append(t[0])
        t = t[1].split("</e2>")
        #print(t)
        e22 = t[0]
        #print('e22: ', e22)

        e2_text = text
        e2_text = " ".join(e2_text)
        e2_text = tokenizer.tokenize(e2_text)
        text.append(t[0])
        text.append(t[1])
        y = tokenizer.tokenize(t[0])
        y[0] +="E22"
        temp.append(" ".join(y))
        temp.append(t[1])

        text = " ".join(text)
        text = tokenizer.tokenize(text)
        temp = " ".join(temp)
        temp = tokenizer.tokenize(temp)

        q1 = tokenizer.tokenize(e11)[0]
        print('q1: ', q1)
        q2 = tokenizer.tokenize(e22)[0]
        print('q2: ', q2)
        
        print('text: ', text)
        
        for i, word in enumerate(text):
            if(word.find(q1)!=-1):
                if(temp[i].find("E11")!=-1):
                    print('e1 is:', i)
                    e1.append(i)            
                    break

        for i, word in enumerate(text):
            if(word.find(q2)!=-1):
                    if(temp[i].find("E22")!=-1):
                        print('e2 is:', i)
                        e2.append(i)   
                        
                    
        text = " ".join(text)
        sentences.append(text)
        #print('sentences is:', text)
        

# for saving training data open "train_data" and for test data open "test_data"
with open('data/relation-3/train_3_data', 'wb') as f:
    pickle.dump((sentences, e1, e2), f)
    f.close()

num:  0
The <e1>island</e1> was farmed for <e2>oats</e2> and barley until 1796 when the owner mandated that oak and alder trees be planted.
q1:  island
q2:  oats
text:  ['The', 'island', 'was', 'farmed', 'for', 'oats', 'and', 'barley', 'until', '1796', 'when', 'the', 'owner', 'mandated', 'that', 'oak', 'and', 'alder', 'trees', 'be', 'planted', '.']
e1 is: 1
e2 is: 5
num:  1
In some Asian countries, including China, Malaysia and the Philippines, the <e1>turtles</e1> are farmed for <e2>food</e2>.
q1:  turtles
q2:  food
text:  ['In', 'some', 'Asian', 'countries', ',', 'including', 'China', ',', 'Malaysia', 'and', 'the', 'Philippines', ',', 'the', 'turtles', 'are', 'farmed', 'for', 'food', '.']
e1 is: 14
e2 is: 18
num:  2
Surrounding each neuron there was a group of cells of the glial type (satellite cells), which together with the <e1>collagen fibres</e1> composed the <e2>glial capsule</e2>.
q1:  collagen
q2:  glial
text:  ['Surrounding', 'each', 'neuron', 'there', 'was', 'a', 'group', 'o

q1:  land
q2:  many
text:  ['It', 'is', 'set', 'in', 'the', 'most', 'beautiful', 'surroundings', 'and', 'the', 'land', 'has', 'been', 'biodynamically', 'farmed', 'for', 'many', 'years', '.']
e1 is: 10
e2 is: 16
num:  21
You're right that the <e1>punk explosion</e1> cultivated the underground/indie rock <e2>scene</e2> that would one day be called 'alternative music' that rooted deep into the culture.
q1:  punk
q2:  scene
text:  ['You', "'re", 'right', 'that', 'the', 'punk', 'explosion', 'cultivated', 'the', 'underground/indie', 'rock', 'scene', 'that', 'would', 'one', 'day', 'be', 'called', '`', 'alternative', 'music', "'", 'that', 'rooted', 'deep', 'into', 'the', 'culture', '.']
e1 is: 5
e2 is: 11
num:  22
Beginning in 1966-67, the <e1>club</e1> made the <e2>playoffs</e2> nine consecutive seasons, the only NHL team of the period to accomplish that feat.
q1:  club
q2:  playoffs
text:  ['Beginning', 'in', '1966-67', ',', 'the', 'club', 'made', 'the', 'playoffs', 'nine', 'consecutive', 's

q1:  experimenters
q2:  shape
text:  ['The', 'experimenters', 'designed', 'the', 'shape', 'and', 'size', 'of', 'the', 'enlarged', 'fiber', 'ends', 'so', 'they', 'do', "n't", 'experience', 'the', 'stresses', 'that', 'usually', 'snap', 'fibers', 'and', 'limit', 'a', 'short-fiber', 'composite', "'s", 'performance', '.']
e1 is: 1
e2 is: 4
num:  44
The third <e1>zone</e1> is procreated by the <e2>mid-ocean ridge</e2>, which is responsible for 7 percent of the earthquakes.
q1:  zone
q2:  mid-ocean
text:  ['The', 'third', 'zone', 'is', 'procreated', 'by', 'the', 'mid-ocean', 'ridge', ',', 'which', 'is', 'responsible', 'for', '7', 'percent', 'of', 'the', 'earthquakes', '.']
e1 is: 2
e2 is: 7
num:  45
Digital computers actually execute numerical instructions: bitstrings that contain information in the 'native' <e1>language</e1> created by the machine's <e2>designers</e2>.
q1:  language
q2:  designers
text:  ['Digital', 'computers', 'actually', 'execute', 'numerical', 'instructions', ':', 'bitst

q1:  statement
q2:  producer
text:  ['The', 'satisfaction', 'was', 'complete', '-', 'please', 'find', 'below', 'a', 'statement', 'from', 'the', 'producer', '.']
e1 is: 9
e2 is: 12
num:  67
He was a <e1>trouble</e1> <e2>maker</e2> then, a leader in SDS, and a trouble maker he remains.
q1:  trouble
q2:  maker
text:  ['He', 'was', 'a', 'trouble', 'maker', 'then', ',', 'a', 'leader', 'in', 'SDS', ',', 'and', 'a', 'trouble', 'maker', 'he', 'remains', '.']
e1 is: 3
e2 is: 4
num:  68
In zen, there is a strong relationship between <e1>work</e1> and <e2>food</e2>, for the former begets the latter.
q1:  work
q2:  food
text:  ['In', 'zen', ',', 'there', 'is', 'a', 'strong', 'relationship', 'between', 'work', 'and', 'food', ',', 'for', 'the', 'former', 'begets', 'the', 'latter', '.']
e1 is: 9
e2 is: 11
num:  69
This <e1>skateboard</e1> <e2>company</e2> designs skateboards specifically for girls.
q1:  skateboard
q2:  company
text:  ['This', 'skateboard', 'company', 'designs', 'skateboards', 'specif

q1:  computer
q2:  laptops
text:  ['The', 'group', 'is', 'alleging', 'the', 'computer', 'giant', 'negligently', 'designed', 'the', 'laptops', ',', 'ultimately', 'causing', 'them', 'to', 'overheat', 'and', 'do', 'damage', 'their', 'own', 'motherboards', '.']
e1 is: 5
e2 is: 10
num:  91
The PI <e1>login</e1> is created by the <e2>administrator</e2>.
q1:  login
q2:  administrator
text:  ['The', 'PI', 'login', 'is', 'created', 'by', 'the', 'administrator', '.']
e1 is: 2
e2 is: 7
num:  92
Other non-native species used in aquaculture are brook trout (Salvelinus fontinalis) and lake trout (Salvelinus namaycush), <e1>fishes</e1> that mainly has been farmed for <e2>stocking purposes</e2>.
q1:  fishes
q2:  stocking
text:  ['Other', 'non-native', 'species', 'used', 'in', 'aquaculture', 'are', 'brook', 'trout', '-LRB-', 'Salvelinus', 'fontinalis', '-RRB-', 'and', 'lake', 'trout', '-LRB-', 'Salvelinus', 'namaycush', '-RRB-', ',', 'fishes', 'that', 'mainly', 'has', 'been', 'farmed', 'for', 'stocking

q1:  tobacco
q2:  producer
text:  ['Yanbian', 'is', 'also', 'blessed', 'with', 'agricultural', 'riches', 'and', 'is', 'a', 'major', 'tobacco', 'producer', '.']
e1 is: 11
e2 is: 12
num:  115
Patient survival one year after transplantation from a living-related donor is 95% and comparably high if the <e1>organ</e1> comes from a cadaveric <e2>donor</e2>.
q1:  organ
q2:  donor
text:  ['Patient', 'survival', 'one', 'year', 'after', 'transplantation', 'from', 'a', 'living-related', 'donor', 'is', '95', '%', 'and', 'comparably', 'high', 'if', 'the', 'organ', 'comes', 'from', 'a', 'cadaveric', 'donor', '.']
e1 is: 18
e2 is: 23
num:  116
However, I believe that this is an opportune moment to change the <e1>political structures</e1> which procreated the <e2>events</e2> of Sept. 11th.
q1:  political
q2:  events
text:  ['However', ',', 'I', 'believe', 'that', 'this', 'is', 'an', 'opportune', 'moment', 'to', 'change', 'the', 'political', 'structures', 'which', 'procreated', 'the', 'events', 'of', '

q1:  farmers
q2:  beans
text:  ['Virtually', 'all', 'hillside', 'farmers', 'grow', 'maize', 'during', 'both', 'seasons', ',', 'and', 'many', 'of', 'the', 'farmers', 'cultivate', 'beans', ',', 'rice', ',', 'and', 'other', 'crops', 'as', 'well', '.']
e1 is: 14
e2 is: 16
num:  136
The <e1>cathode</e1> emits <e2>electrons</e2>.
q1:  cathode
q2:  electrons
text:  ['The', 'cathode', 'emits', 'electrons', '.']
e1 is: 1
e2 is: 3
num:  137
The lower mark is the Vienna <e1>company</e1> that made the <e2>mug</e2>.
q1:  company
q2:  mug
text:  ['The', 'lower', 'mark', 'is', 'the', 'Vienna', 'company', 'that', 'made', 'the', 'mug', '.']
e1 is: 6
e2 is: 10
num:  138
Peterson worked as a <e1>fertilizer</e1> <e2>salesman</e2> before his arrest.
q1:  fertilizer
q2:  salesman
text:  ['Peterson', 'worked', 'as', 'a', 'fertilizer', 'salesman', 'before', 'his', 'arrest', '.']
e1 is: 4
e2 is: 5
num:  139
The <e1>headmaster</e1> made the formal <e2>announcement</e2>.
q1:  headmaster
q2:  announcement
text:  

### To extract the SDPs and get word / POS / dependency types

In [1107]:
os.environ['CLASSPATH'] = r'/usr/local/Cellar/stanford-parser/3.9.1/libexec'

#sentence = 'The brown fox is quick and he is jumping over the lazy dog'

dep_parser = StanfordDependencyParser()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.StanforCoreNLPDependencyParser instead.
  """


In [1108]:
def lca(tree, index1, index2):
    node = index1
    path1 = []
    path2 = []
    path1.append(index1)
    path2.append(index2)
    while(node != tree.root):
        node = tree.nodes[node['head']]
        path1.append(node)
    node = index2
    while(node != tree.root):
        node = tree.nodes[node['head']]
        path2.append(node)
    for l1, l2 in zip(path1[::-1],path2[::-1]):
        if(l1==l2):
            temp = l1
    return temp


def path_lca(tree, node, lca_node):
    path = []
    path.append(node)
    while(node != lca_node):
        node = tree.nodes[node['head']]
        path.append(node)
    return path

In [1109]:
f = open('data/relation-3/train_3_data', 'rb')
sentences, e1, e2 = pickle.load(f)

#print(sentences[:10])
#print('e1:', e1[0:10])
#print('e2:', e2[:10])
f.close()


word_path1 = []
word_path2 = []
rel_path1 = []
rel_path2 = []
pos_path1 = []
pos_path2 = []
for i in range(140):
    word_path1.append(0)
    word_path2.append(0)
    rel_path1.append(0)
    rel_path2.append(0)
    pos_path1.append(0)
    pos_path2.append(0)
    
#print("word_path1", word_path1)
#print("rel_path1", rel_path1)



for i in range(140):
    try:
        parse_tree = dep_parser.raw_parse(sentences[i])
        for trees in parse_tree:
            tree = trees
        node1 = tree.nodes[e1[i]+1]
        node2 = tree.nodes[e2[i]+1]
        if node1['address']!=None and node2['address']!=None:
            print(i, "success")
            lca_node = lca(tree, node1, node2)
            print('lca: ', lca_node)
            #print('\n')
            path1 = path_lca(tree, node1, lca_node)
            path2 = path_lca(tree, node2, lca_node)
            print('path1: ', path1)
            print('path2: ', path2)
            #print('\n')
            word_path1[i] = [p["word"] for p in path1]
            word_path2[i] = [p["word"] for p in path2]
            print('word_path1: ', word_path1)
            print('word_path2: ', word_path2)
            #print('\n')
            rel_path1[i] = [p["rel"] for p in path1]
            rel_path2[i] = [p["rel"] for p in path2]
            print('rel_path1: ', rel_path1)
            print('rel_path2: ', rel_path2)
            #print('\n')
            pos_path1[i] = [p["tag"] for p in path1]
            pos_path2[i] = [p["tag"] for p in path2]
            print('pos_path1: ',  pos_path1)
            print('pos_path2: ',  pos_path2)
            print('\n')
        else:
            print(i, node1["address"], node2["address"])
    except AssertionError:
        print(i, "error")
    

0 success
lca:  {'address': 4, 'word': 'farmed', 'lemma': '_', 'ctag': 'VBN', 'tag': 'VBN', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubjpass': [2], 'auxpass': [3], 'nmod': [6, 10], 'advcl': [14]}), 'rel': 'root'}
path1:  [{'address': 2, 'word': 'island', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 4, 'deps': defaultdict(<class 'list'>, {'det': [1]}), 'rel': 'nsubjpass'}, {'address': 4, 'word': 'farmed', 'lemma': '_', 'ctag': 'VBN', 'tag': 'VBN', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubjpass': [2], 'auxpass': [3], 'nmod': [6, 10], 'advcl': [14]}), 'rel': 'root'}]
path2:  [{'address': 6, 'word': 'oats', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 4, 'deps': defaultdict(<class 'list'>, {'case': [5], 'cc': [7], 'conj': [8]}), 'rel': 'nmod'}, {'address': 4, 'word': 'farmed', 'lemma': '_', 'ctag': 'VBN', 'tag': 'VBN', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubjpass': [2], 'auxpas

3 success
lca:  {'address': 10, 'word': 'provide', 'lemma': '_', 'ctag': 'VBP', 'tag': 'VBP', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [3], 'dobj': [13]}), 'rel': 'root'}
path1:  [{'address': 6, 'word': 'tones', 'lemma': '_', 'ctag': 'NNS', 'tag': 'NNS', 'feats': '_', 'head': 3, 'deps': defaultdict(<class 'list'>, {'amod': [5], 'nmod': [9]}), 'rel': 'conj'}, {'address': 3, 'word': 'brown', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 10, 'deps': defaultdict(<class 'list'>, {'det': [1], 'amod': [2], 'cc': [4], 'conj': [6]}), 'rel': 'nsubj'}, {'address': 10, 'word': 'provide', 'lemma': '_', 'ctag': 'VBP', 'tag': 'VBP', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [3], 'dobj': [13]}), 'rel': 'root'}]
path2:  [{'address': 12, 'word': 'tranquil', 'lemma': '_', 'ctag': 'JJ', 'tag': 'JJ', 'feats': '_', 'head': 13, 'deps': defaultdict(<class 'list'>, {}), 'rel': 'amod'}, {'address': 13, 'word': 'mood', 'lemma': '_', 'cta

5 success
lca:  {'address': 3, 'word': 'fissure', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 1, 'deps': defaultdict(<class 'list'>, {'det': [2], 'acl': [4]}), 'rel': 'dobj'}
path1:  [{'address': 3, 'word': 'fissure', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 1, 'deps': defaultdict(<class 'list'>, {'det': [2], 'acl': [4]}), 'rel': 'dobj'}]
path2:  [{'address': 7, 'word': 'meteor', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 9, 'deps': defaultdict(<class 'list'>, {'det': [6], 'case': [8]}), 'rel': 'nmod:poss'}, {'address': 9, 'word': 'impact', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 4, 'deps': defaultdict(<class 'list'>, {'case': [5], 'nmod:poss': [7]}), 'rel': 'nmod'}, {'address': 4, 'word': 'created', 'lemma': '_', 'ctag': 'VBN', 'tag': 'VBN', 'feats': '_', 'head': 3, 'deps': defaultdict(<class 'list'>, {'nmod': [9]}), 'rel': 'acl'}, {'address': 3, 'word': 'fissure', 'lemma': '_', 'ctag': 'NN', 'tag': 'N

7 success
lca:  {'address': 20, 'word': 'provide', 'lemma': '_', 'ctag': 'VBP', 'tag': 'VBP', 'feats': '_', 'head': 11, 'deps': defaultdict(<class 'list'>, {'nsubj': [19], 'xcomp': [24]}), 'rel': 'conj'}
path1:  [{'address': 19, 'word': 'waiters', 'lemma': '_', 'ctag': 'NNS', 'tag': 'NNS', 'feats': '_', 'head': 20, 'deps': defaultdict(<class 'list'>, {'det': [18]}), 'rel': 'nsubj'}, {'address': 20, 'word': 'provide', 'lemma': '_', 'ctag': 'VBP', 'tag': 'VBP', 'feats': '_', 'head': 11, 'deps': defaultdict(<class 'list'>, {'nsubj': [19], 'xcomp': [24]}), 'rel': 'conj'}]
path2:  [{'address': 22, 'word': 'show', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 24, 'deps': defaultdict(<class 'list'>, {'det': [21]}), 'rel': 'nsubj'}, {'address': 24, 'word': 'watching', 'lemma': '_', 'ctag': 'VBG', 'tag': 'VBG', 'feats': '_', 'head': 20, 'deps': defaultdict(<class 'list'>, {'nsubj': [22], 'dep': [23]}), 'rel': 'xcomp'}, {'address': 20, 'word': 'provide', 'lemma': '_', 'ctag': 'V

9 success
lca:  {'address': 7, 'word': 'cultivated', 'lemma': '_', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nmod': [3], 'nsubj': [6], 'dobj': [9], 'cc': [13], 'conj': [18]}), 'rel': 'root'}
path1:  [{'address': 6, 'word': 'judges', 'lemma': '_', 'ctag': 'NNS', 'tag': 'NNS', 'feats': '_', 'head': 7, 'deps': defaultdict(<class 'list'>, {'det': [5]}), 'rel': 'nsubj'}, {'address': 7, 'word': 'cultivated', 'lemma': '_', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nmod': [3], 'nsubj': [6], 'dobj': [9], 'cc': [13], 'conj': [18]}), 'rel': 'root'}]
path2:  [{'address': 9, 'word': 'principles', 'lemma': '_', 'ctag': 'NNS', 'tag': 'NNS', 'feats': '_', 'head': 7, 'deps': defaultdict(<class 'list'>, {'det': [8], 'nmod': [11]}), 'rel': 'dobj'}, {'address': 7, 'word': 'cultivated', 'lemma': '_', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nmod': [3], 'nsub

...

135 success
lca:  {'address': 16, 'word': 'cultivate', 'lemma': '_', 'ctag': 'VBP', 'tag': 'VBP', 'feats': '_', 'head': 5, 'deps': defaultdict(<class 'list'>, {'nsubj': [12], 'dobj': [17], 'advmod': [24]}), 'rel': 'conj'}
path1:  [{'address': 15, 'word': 'farmers', 'lemma': '_', 'ctag': 'NNS', 'tag': 'NNS', 'feats': '_', 'head': 12, 'deps': defaultdict(<class 'list'>, {'case': [13], 'det': [14]}), 'rel': 'nmod'}, {'address': 12, 'word': 'many', 'lemma': '_', 'ctag': 'JJ', 'tag': 'JJ', 'feats': '_', 'head': 16, 'deps': defaultdict(<class 'list'>, {'nmod': [15]}), 'rel': 'nsubj'}, {'address': 16, 'word': 'cultivate', 'lemma': '_', 'ctag': 'VBP', 'tag': 'VBP', 'feats': '_', 'head': 5, 'deps': defaultdict(<class 'list'>, {'nsubj': [12], 'dobj': [17], 'advmod': [24]}), 'rel': 'conj'}]
path2:  [{'address': 17, 'word': 'beans', 'lemma': '_', 'ctag': 'NNS', 'tag': 'NNS', 'feats': '_', 'head': 16, 'deps': defaultdict(<class 'list'>, {'conj': [19, 23], 'cc': [21]}), 'rel': 'dobj'}, {'address': 1

136 success
lca:  {'address': 3, 'word': 'emits', 'lemma': '_', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [2], 'dobj': [4]}), 'rel': 'root'}
path1:  [{'address': 2, 'word': 'cathode', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 3, 'deps': defaultdict(<class 'list'>, {'det': [1]}), 'rel': 'nsubj'}, {'address': 3, 'word': 'emits', 'lemma': '_', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [2], 'dobj': [4]}), 'rel': 'root'}]
path2:  [{'address': 4, 'word': 'electrons', 'lemma': '_', 'ctag': 'NNS', 'tag': 'NNS', 'feats': '_', 'head': 3, 'deps': defaultdict(<class 'list'>, {}), 'rel': 'dobj'}, {'address': 3, 'word': 'emits', 'lemma': '_', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [2], 'dobj': [4]}), 'rel': 'root'}]
word_path1:  [['island', 'farmed'], ['turtles', 'farmed'], ['collagen', 'fibres', 'togethe

137 success
lca:  {'address': 7, 'word': 'company', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [3], 'cop': [4], 'det': [5], 'compound': [6], 'acl:relcl': [9]}), 'rel': 'root'}
path1:  [{'address': 7, 'word': 'company', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [3], 'cop': [4], 'det': [5], 'compound': [6], 'acl:relcl': [9]}), 'rel': 'root'}]
path2:  [{'address': 11, 'word': 'mug', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 9, 'deps': defaultdict(<class 'list'>, {'det': [10]}), 'rel': 'dobj'}, {'address': 9, 'word': 'made', 'lemma': '_', 'ctag': 'VBD', 'tag': 'VBD', 'feats': '_', 'head': 7, 'deps': defaultdict(<class 'list'>, {'nsubj': [8], 'dobj': [11]}), 'rel': 'acl:relcl'}, {'address': 7, 'word': 'company', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [3], 'cop':

138 success
lca:  {'address': 6, 'word': 'salesman', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 2, 'deps': defaultdict(<class 'list'>, {'case': [3], 'det': [4], 'compound': [5]}), 'rel': 'nmod'}
path1:  [{'address': 5, 'word': 'fertilizer', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 6, 'deps': defaultdict(<class 'list'>, {}), 'rel': 'compound'}, {'address': 6, 'word': 'salesman', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 2, 'deps': defaultdict(<class 'list'>, {'case': [3], 'det': [4], 'compound': [5]}), 'rel': 'nmod'}]
path2:  [{'address': 6, 'word': 'salesman', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 2, 'deps': defaultdict(<class 'list'>, {'case': [3], 'det': [4], 'compound': [5]}), 'rel': 'nmod'}]
word_path1:  [['island', 'farmed'], ['turtles', 'farmed'], ['collagen', 'fibres', 'together', 'composed'], ['tones', 'brown', 'provide'], ['usher', 'prepares'], ['fissure'], ['artist', 'made'], ['waiters', '

139 success
lca:  {'address': 3, 'word': 'made', 'lemma': '_', 'ctag': 'VBD', 'tag': 'VBD', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [2], 'dobj': [6]}), 'rel': 'root'}
path1:  [{'address': 2, 'word': 'headmaster', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 3, 'deps': defaultdict(<class 'list'>, {'det': [1]}), 'rel': 'nsubj'}, {'address': 3, 'word': 'made', 'lemma': '_', 'ctag': 'VBD', 'tag': 'VBD', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [2], 'dobj': [6]}), 'rel': 'root'}]
path2:  [{'address': 6, 'word': 'announcement', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 3, 'deps': defaultdict(<class 'list'>, {'det': [4], 'amod': [5]}), 'rel': 'dobj'}, {'address': 3, 'word': 'made', 'lemma': '_', 'ctag': 'VBD', 'tag': 'VBD', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [2], 'dobj': [6]}), 'rel': 'root'}]
word_path1:  [['island', 'farmed'], ['turtles', 'farmed'], ['colla

In [1075]:
file = open('data/relation-3/train_3_paths', 'wb')
pickle.dump([word_path1, word_path2, rel_path1, rel_path2, pos_path1, pos_path2], file)

In [1110]:
f = open(data_dir + '/relation-1/vocab.pkl', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"
word2id[unknown_token] = word_vocab_size -1
id2word[word_vocab_size-1] = unknown_token

pos_tags_vocab = []
for line in open(data_dir + '/relation-1/pos_tags.txt'):
        pos_tags_vocab.append(line.strip())
print('pos_tags_vocab: ', pos_tags_vocab)

dep_vocab = []
for line in open(data_dir + '/relation-1/dependency_types.txt'):
    dep_vocab.append(line.strip())
print('dep_vocab: ', dep_vocab)

relation_vocab = []
for line in open(data_dir + '/relation-3/relation_types.txt'):
    relation_vocab.append(line.strip())
print('relation_vocab: ', relation_vocab)

rel2id = dict((w, i) for i,w in enumerate(relation_vocab))
id2rel = dict((i, w) for i,w in enumerate(relation_vocab))
print('rel2id: ', rel2id)

pos_tag2id = dict((w, i) for i,w in enumerate(pos_tags_vocab))
id2pos_tag = dict((i, w) for i,w in enumerate(pos_tags_vocab))
print('pos_tag2id: ', pos_tag2id)

dep2id = dict((w, i) for i,w in enumerate(dep_vocab))
id2dep = dict((i, w) for i,w in enumerate(dep_vocab))
print('dep2id: ', dep2id)

pos_tag2id['OTH'] = 9
id2pos_tag[9] = 'OTH'

dep2id['OTH'] = 20
id2dep[20] = 'OTH'

JJ_pos_tags = ['JJ', 'JJR', 'JJS']
NN_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS']
RB_pos_tags = ['RB', 'RBR', 'RBS']
PRP_pos_tags = ['PRP', 'PRP$']
VB_pos_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
_pos_tags = ['CC', 'CD', 'DT', 'IN']

def pos_tag(x):
    if x in JJ_pos_tags:
        return pos_tag2id['JJ']
    if x in NN_pos_tags:
        return pos_tag2id['NN']
    if x in RB_pos_tags:
        return pos_tag2id['RB']
    if x in PRP_pos_tags:
        return pos_tag2id['PRP']
    if x in VB_pos_tags:
        return pos_tag2id['VB']
    if x in _pos_tags:
        return pos_tag2id[x]
    else:
        return 9

pos_tags_vocab:  ['CC', 'CD', 'DT', 'IN', 'JJ', 'NN', 'PRP', 'RB', 'VB']
dep_vocab:  ['root', 'nmod', 'nsubj', 'dobj', 'nsubjpass', 'compound', 'conj', 'acl', 'advcl', 'ccomp', 'amod', 'acl:relcl', 'xcomp', 'dep', 'appos', 'nmod:poss', 'advmod', 'parataxis', 'csubj', 'iobj']
relation_vocab:  ['Product-Producer(e1,e2)', 'Product-Producer(e2,e1)', 'others']
rel2id:  {'Product-Producer(e1,e2)': 0, 'Product-Producer(e2,e1)': 1, 'others': 2}
pos_tag2id:  {'CC': 0, 'CD': 1, 'DT': 2, 'IN': 3, 'JJ': 4, 'NN': 5, 'PRP': 6, 'RB': 7, 'VB': 8}
dep2id:  {'root': 0, 'nmod': 1, 'nsubj': 2, 'dobj': 3, 'nsubjpass': 4, 'compound': 5, 'conj': 6, 'acl': 7, 'advcl': 8, 'ccomp': 9, 'amod': 10, 'acl:relcl': 11, 'xcomp': 12, 'dep': 13, 'appos': 14, 'nmod:poss': 15, 'advmod': 16, 'parataxis': 17, 'csubj': 18, 'iobj': 19}


### LSTM Model

In [1129]:
import sys, os, _pickle as pickle
import tensorflow as tf
import numpy as np
import nltk
from sklearn.metrics import f1_score

In [1130]:
data_dir = 'data'                        # Directory for Data and Other files
ckpt_dir = 'data/relation-3/checkpoint'                  # Directory for Checkpoints 
word_embd_dir = 'data/relation-3/checkpoint/word_embd'   # Directory for Checkpoints of Word Embedding Layer
model_dir = 'data/relation-3/checkpoint/relation-3-model'         # Directory for Checkpoints of Model

In [1131]:
word_embd_dim = 100       # Dimension of embedding layer for words
pos_embd_dim = 25         # Dimension of embedding layer for POS Tags
dep_embd_dim = 25         # Dimension of embedding layer for Dependency Types

word_vocab_size = 400001 #400001  # Vocab size for Words
pos_vocab_size = 10       # Vocab size for POS Tags
dep_vocab_size = 21       # Vocab size for Dependency Types

relation_classes = 3      # No. of Relation Classes
state_size = 100          # Dimension of States of LSTM-RNNs
batch_size = 1        # Batch Size for training

word_state_size = 100
other_state_size = 100
channels = 3              # No. of types of features to feed in LSTM
lambda_l2 = 0.0001
max_len_path = 10         # Maximum length of sequence

In [1132]:
tf.reset_default_graph()

with tf.Graph().as_default():
    
    # Length of the sequence
    path_length = tf.placeholder(tf.int32, shape=[2,None], name="path1_length") 
    
    # Words in the sequence
    word_ids = tf.placeholder(tf.int32, shape=[2,None, max_len_path], name="word_ids") 
    
     # POS Tags in the sequence
    pos_ids = tf.placeholder(tf.int32, [2,None, max_len_path], name="pos_ids") 
    
    # Dependency Types in the sequence
    dep_ids = tf.placeholder(tf.int32, [2,None, max_len_path], name="dep_ids") 
    
     # True Relation btw the entities
    y = tf.placeholder(tf.int32, [None], name="y")   



In [1133]:
with tf.name_scope("input"):
    
    # Length of the sequence
    path_length = tf.placeholder(tf.int32, shape=[2, None], name="path1_length") 
    
    # Words in the sequence
    word_ids = tf.placeholder(tf.int32, shape=[2, None, max_len_path], name="word_ids") 
    
     # POS Tags in the sequence
    pos_ids = tf.placeholder(tf.int32, [2, None, max_len_path], name="pos_ids") 
    
    # Dependency Types in the sequence
    dep_ids = tf.placeholder(tf.int32, [2, None, max_len_path], name="dep_ids") 
    
     # True Relation btw the entities
    y = tf.placeholder(tf.int32, [None], name="y")   

In [1134]:
# Embedding Layer of Words 
with tf.name_scope("word_embedding"):
    W = tf.Variable(tf.constant(0.0, shape=[word_vocab_size, word_embd_dim]), name="W")
    embedding_placeholder = tf.placeholder(tf.float32,[word_vocab_size, word_embd_dim])
    embedding_init = W.assign(embedding_placeholder)
    embedded_word = tf.nn.embedding_lookup(W, word_ids)
    word_embedding_saver = tf.train.Saver({"word_embedding/W": W})

# Embedding Layer of POS Tags 
with tf.name_scope("pos_embedding"):
    W = tf.Variable(tf.random_uniform([pos_vocab_size, pos_embd_dim]), name="W")
    embedded_pos = tf.nn.embedding_lookup(W, pos_ids)
    pos_embedding_saver = tf.train.Saver({"pos_embedding/W": W})

# Embedding Layer of Dependency Types 
with tf.name_scope("dep_embedding"):
    W = tf.Variable(tf.random_uniform([dep_vocab_size, dep_embd_dim]), name="W")
    embedded_dep = tf.nn.embedding_lookup(W, dep_ids)
    dep_embedding_saver = tf.train.Saver({"dep_embedding/W": W})

In [1135]:
hidden_states = tf.zeros([channels, batch_size, state_size], name="hidden_state")
cell_states = tf.zeros([channels, batch_size, state_size], name="cell_state")

init_states = [tf.contrib.rnn.LSTMStateTuple(hidden_states[i], cell_states[i]) for i in range(channels)]

with tf.variable_scope("word_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word[0], sequence_length=path_length[0], initial_state=init_states[0])
    state_series_word1 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("word_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word[1], sequence_length=path_length[1], initial_state=init_states[0])
    state_series_word2 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("pos_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[0], sequence_length=path_length[0],initial_state=init_states[1])
    state_series_pos1 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("pos_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[1], sequence_length=path_length[1],initial_state=init_states[1])
    state_series_pos2 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("dep_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[0], sequence_length=path_length[0], initial_state=init_states[2])
    state_series_dep1 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("dep_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[1], sequence_length=path_length[1], initial_state=init_states[2])
    state_series_dep2 = tf.reduce_max(state_series, axis=1)

state_series1 = tf.concat([state_series_word1, state_series_pos1, state_series_dep1], 1)
state_series2 = tf.concat([state_series_word2, state_series_pos2, state_series_dep2], 1)

state_series = tf.concat([state_series1, state_series2], 1)

In [1136]:
with tf.name_scope("hidden_layer"):
    W = tf.Variable(tf.truncated_normal([600, 100], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([100]), name="b")
    y_hidden_layer = tf.matmul(state_series, W) + b

with tf.name_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([100, relation_classes], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([relation_classes]), name="b")
    logits = tf.matmul(y_hidden_layer, W) + b
    predictions = tf.argmax(logits, 1)

In [1137]:
with tf.name_scope("loss"):
    #l2_loss = lambda_l2 * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    total_loss = loss # + l2_loss

with tf.variable_scope('train'):
    global_step = tf.Variable(0, name="global_step")
    optimizer = tf.train.AdamOptimizer(0.001).minimize(total_loss, global_step=global_step)

    
with tf.variable_scope('validation'):
    valid_step = tf.Variable(0, name="valid_step")
    optimizer_valid = tf.train.AdamOptimizer(0.001).minimize(total_loss, global_step=valid_step)


In [1138]:
relations = []
for line in open(data_dir + '/relation-3/relation_3_train.txt'):
    #print(line.strip().split()[0])
    relations.append(line.strip().split())    
print('relations', len(relations[0]))

print(relations[1][0])

relations 1
others


In [1139]:
f = open(data_dir + '/relation-3/train_3_paths', 'rb')
word_p1, word_p2, dep_p1, dep_p2, pos_p1, pos_p2 = pickle.load(f)
print(len(dep_p1))
print('word_p1:', word_p1)
f.close()

length = len(word_p1)
print(length)


num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p1[i]):
        word = word.lower()
        word_p1[i][j] = word if word in word2id else unknown_token 
    for k, word in enumerate(word_p2[i]):
        word = word.lower()
        word_p2[i][k] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p1[i]):
        dep_p1[i][l] = d if d in dep2id else 'OTH'
    for m, d in enumerate(dep_p2[i]):
        dep_p2[i][m] = d if d in dep2id else 'OTH'

        
word_p1_ids = np.ones([length, max_len_path],dtype=int)
word_p2_ids = np.ones([length, max_len_path],dtype=int)
pos_p1_ids = np.ones([length, max_len_path],dtype=int)
pos_p2_ids = np.ones([length, max_len_path],dtype=int)
dep_p1_ids = np.ones([length, max_len_path],dtype=int)
dep_p2_ids = np.ones([length, max_len_path],dtype=int)


#rel_ids = np.array([rel2id[rel for rel in relations])
rel_ids = np.array([rel2id[rel[0]] for rel in relations])
print(len(rel_ids))
print(rel_ids)


path1_len = np.array([len(w) for w in word_p1], dtype=int)
path2_len = np.array([len(w) for w in word_p2])

for i in range(length):
    for j, w in enumerate(word_p1[i]):
        word_p1_ids[i][j] = word2id[w]
    for j, w in enumerate(word_p2[i]):
        word_p2_ids[i][j] = word2id[w]
    for j, w in enumerate(pos_p1[i]):
        pos_p1_ids[i][j] = pos_tag(w)
    for j, w in enumerate(pos_p2[i]):
        pos_p2_ids[i][j] = pos_tag(w)
    for j, w in enumerate(dep_p1[i]):
        dep_p1_ids[i][j] = dep2id[w]
    for j, w in enumerate(dep_p2[i]):
        dep_p2_ids[i][j] = dep2id[w]


140
word_p1: [['island', 'farmed'], ['turtles', 'farmed'], ['collagen', 'fibres', 'together', 'composed'], ['tones', 'brown', 'provide'], ['usher', 'prepares'], ['fissure'], ['artist', 'made'], ['waiters', 'provide'], ['faculty', 'designed'], ['judges', 'cultivated'], ['simulation', 'study', 'produces'], ['grass', 'farmed'], ['hydrogen', 'comes'], ['opium', 'exporter'], ['trucking', 'company'], ['map', 'maker'], ['scientists', 'made'], ['oceans', 'farmed'], ['boy'], ['router', 'manufacturer'], ['land', 'farmed'], ['punk', 'explosion', 'cultivated'], ['club', 'made'], ['vaccine', 'produced'], ['attacker', 'developed'], ['land', 'farmed'], ['students'], ['tram', 'makes'], ['smoothie', 'maker'], ['emission', 'produced'], ['furniture', 'maker'], ['artists'], ['defamation', 'comes'], ['monitor', 'provide'], ['tutors', 'provide'], ['growth', 'hormone'], ['guy'], ['site', 'farmed'], ['crisis', 'begets'], ['jobs'], ['antihelion', 'radiant', 'produced'], ['noise', 'most', 'comes'], ['false', 'm

In [1140]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [1141]:
valid_path_dict = [path1_len[(num_batches-1)*batch_size:num_batches*batch_size], path2_len[(num_batches-1)*batch_size:num_batches*batch_size]]
valid_word_dict = [word_p1_ids[(num_batches-1)*batch_size:num_batches*batch_size], word_p2_ids[(num_batches-1)*batch_size:num_batches*batch_size]]
valid_pos_dict = [pos_p1_ids[(num_batches-1)*batch_size:num_batches*batch_size], pos_p2_ids[(num_batches-1)*batch_size:num_batches*batch_size]]
valid_dep_dict = [dep_p1_ids[(num_batches-1)*batch_size:num_batches*batch_size], dep_p2_ids[(num_batches-1)*batch_size:num_batches*batch_size]]
valid_y_dict = rel_ids[(num_batches-1)*batch_size:(num_batches)*batch_size]  #.reshape(10,)
#print('y_dict: ', y_dict)
print(np.shape(valid_path_dict), '\n', np.shape(valid_word_dict), '\n', np.shape(valid_pos_dict), '\n', np.shape(valid_dep_dict), '\n', np.shape(valid_y_dict))
valid_feed_dict = {
    path_length:valid_path_dict,
    word_ids:valid_word_dict,
    pos_ids:valid_pos_dict,
    dep_ids:valid_dep_dict,
    y:valid_y_dict}
#print('valid_y_dict: ', valid_y_dict, 'len_valid_y_dict: ', len(valid_y_dict))
#print('valid_word_ids: ', valid_word_dict, 'len_valid_word_ids: ', np.shape(valid_word_dict))

(2, 1) 
 (2, 1, 10) 
 (2, 1, 10) 
 (2, 1, 10) 
 (1,)


In [1142]:
loss_all=[]
num_epochs = 7

for i in range(num_epochs):
    print(num_batches)
    for j in range(num_batches-1):
        print(j*batch_size, (j+1)*batch_size)
        path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
        word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
        pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
        dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
        y_dict = rel_ids[j*batch_size:(j+1)*batch_size]  #.reshape(10,)
        #print('y_dict: ', y_dict)
        
        #print(np.shape(path_dict), '\n', np.shape(word_dict), '\n', np.shape(pos_dict), '\n', np.shape(dep_dict), '\n', np.shape(y_dict))

        feed_dict = {
            path_length:path_dict,
            word_ids:word_dict,
            pos_ids:pos_dict,
            dep_ids:dep_dict,
            y:y_dict}
        
        _, loss, step = sess.run([optimizer, total_loss, global_step], feed_dict)
        
        loss_all.append(loss)
        
        if step%5==0:
            print("Step:", step, "loss:", loss)
    
        if step % 13 == 0:
            saver.save(sess, model_dir + '/model')
            
            __, valid_loss = sess.run([optimizer, total_loss], valid_feed_dict)
            print('The valid loss is: {}'.format(valid_loss))
            
            print("Saved Model")

140
0 1
1 2
2 3
3 4
4 5
Step: 5 loss: 0.9564736
5 6
6 7
7 8
8 9
9 10
Step: 10 loss: 1.1361276
10 11
11 12
12 13
The valid loss is: 0.9907686710357666
Saved Model
13 14
Step: 15 loss: 0.59749544
14 15
15 16
16 17
17 18
18 19
Step: 20 loss: 0.94617003
19 20
20 21
21 22
22 23
23 24
Step: 25 loss: 2.7209134
24 25
The valid loss is: 0.8981269001960754
Saved Model
25 26
26 27
27 28
Step: 30 loss: 0.75466675
28 29
29 30
30 31
31 32
32 33
Step: 35 loss: 1.9386146
33 34
34 35
35 36
36 37
The valid loss is: 0.9417107701301575
Saved Model
37 38
38 39
39 40
40 41
41 42
Step: 45 loss: 1.5808291
42 43
43 44
44 45
45 46
46 47
Step: 50 loss: 0.9380182
47 48
48 49
The valid loss is: 1.0315520763397217
Saved Model
49 50
50 51
Step: 55 loss: 1.1086388
51 52
52 53
53 54
54 55
55 56
Step: 60 loss: 1.0001997
56 57
57 58
58 59
59 60
60 61
Step: 65 loss: 1.1085849
The valid loss is: 0.9423018097877502
Saved Model
61 62
62 63
63 64
64 65
Step: 70 loss: 1.1717958
65 66
66 67
67 68
68 69
69 70
Step: 75 loss: 1.1

In [1143]:
f = open('data/relation-3/loss.txt', 'w') 
for i in loss_all:
    f.write(str(i)+'\n')

In [1144]:
# training accuracy
all_predictions = []
for j in range(num_batches-1):
    path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
    word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
    pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
    dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
    y_dict = rel_ids[j*batch_size:(j+1)*batch_size]

    feed_dict = {
        path_length:path_dict,
        word_ids:word_dict,
        pos_ids:pos_dict,
        dep_ids:dep_dict,
        y:y_dict}
    
    batch_predictions = sess.run(predictions, feed_dict)
    
    all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches-1):
    for pred in all_predictions[i]:
        y_pred.append(pred)
print(y_pred)
print(len(y_pred))


print(rel_ids)
print(len(rel_ids))


count = 0
for i in range(batch_size*(num_batches-1)):
    count += (y_pred[i]==rel_ids[i])
accuracy = count/(batch_size*(num_batches)) * 100

print("training accuracy", accuracy, '\n')


[1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 0, 0, 1, 2, 1, 0, 2, 2, 2, 0, 1, 1, 2, 2, 0, 2, 0, 1, 0, 1, 2, 0, 1, 2, 2, 2, 2, 0, 0, 1, 0, 0, 2, 2, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 2, 1, 1, 1, 0, 2, 0, 2, 2, 0, 1, 1, 1, 0, 1, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 0, 2, 2, 0, 2, 1, 2, 2, 0, 2, 2, 0, 1, 1, 0, 1, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 0, 2, 1, 1, 0, 1, 2, 2, 1, 0, 1, 1, 1, 2]
139
[1 2 2 2 2 2 1 1 1 1 1 2 2 2 2 0 1 2 1 0 2 2 2 0 1 1 2 2 0 2 0 1 0 1 2 0 1
 2 2 2 2 0 0 1 0 0 2 2 1 0 1 1 1 0 0 1 0 0 1 1 2 1 1 1 0 2 0 2 2 0 1 1 1 0
 1 1 0 2 1 2 1 2 1 2 1 2 2 1 2 2 2 0 2 2 0 2 1 2 2 0 2 2 0 1 1 0 1 2 2 0 2
 0 2 0 0 2 2 1 1 1 1 1 2 1 2 0 2 1 1 0 1 2 2 1 0 1 1 1 2 1]
140
training accuracy 98.57142857142858 



In [1145]:
# validation accuracy
valid_predictions = []
valid_batch_predictions = sess.run(predictions, valid_feed_dict)

valid_predictions.append(valid_batch_predictions)
#print('valid_predictions: ', valid_predictions)

valid_y_pred = []
for pred in valid_predictions[0]:
    valid_y_pred.append(pred)
#print('valid_y_pred: ', valid_y_pred)
#print('true valid label: ', rel_ids[130:139])

count = 0
for i in range(batch_size):
    count += (valid_y_pred[i]==rel_ids[i+batch_size*(num_batches-1)])
accuracy = count/(batch_size) * 100

print("validation accuracy", accuracy)

validation accuracy 100.0


### Data pre-processing and SDPs extraction on test dataset

In [1146]:
lines, rels = [], []
for line in open("test/relation-3-test.txt",  encoding="latin"):
    lines.append(line.strip())
    
sentences = []
e1 = []
e2 = []
i = 0

num = 0
for i,w in enumerate(lines):
    m = re.match(r'^([0-9]+)\s"(.+)"$', w.strip())
    
    if(m is not None):
        print('num: ', num)
        num+=1
        temporary = m.group(2)
        print(temporary)
    
    
        text = []
        temp = []
        t = temporary.split("<e1>")
        text.append(t[0])
        #print('text: ', text)

        temp.append(t[0])

        t = t[1].split("</e1>")
        e1_text = text
        e1_text = " ".join(e1_text)
        e1_text = tokenizer.tokenize(e1_text)
        #print('e1_text: ', e1_text)

        text.append(t[0])
        #print('text: ', text)

        e11= t[0]
        #print('e11: ', e11)


        y = tokenizer.tokenize(t[0])
        #print(y)
        y[0] +="E11"

        temp.append(" ".join(y))
        t = t[1].split("<e2>")
        #print(t)
        text.append(t[0])
        temp.append(t[0])
        t = t[1].split("</e2>")
        #print(t)
        e22 = t[0]
        #print('e22: ', e22)

        e2_text = text
        e2_text = " ".join(e2_text)
        e2_text = tokenizer.tokenize(e2_text)
        text.append(t[0])
        text.append(t[1])
        y = tokenizer.tokenize(t[0])
        y[0] +="E22"
        temp.append(" ".join(y))
        temp.append(t[1])

        text = " ".join(text)
        text = tokenizer.tokenize(text)
        temp = " ".join(temp)
        temp = tokenizer.tokenize(temp)

        q1 = tokenizer.tokenize(e11)[0]
        print('q1: ', q1)
        q2 = tokenizer.tokenize(e22)[0]
        print('q2: ', q2)
        
        print('text: ', text)
        
        for i, word in enumerate(text):
            if(word.find(q1)!=-1):
                if(temp[i].find("E11")!=-1):
                    print('e1 is:', i)
                    e1.append(i)            
                    break

        for i, word in enumerate(text):
            if(word.find(q2)!=-1):
                    if(temp[i].find("E22")!=-1):
                        print('e2 is:', i)
                        e2.append(i)   
                        
                    
        text = " ".join(text)
        sentences.append(text)
        #print('sentences is:', text)



num:  0
The <e1>introduction</e1> of the gas stove, the electric mixer and the food processor made the <e2>process</e2> easier, faster and more reliable.
q1:  introduction
q2:  process
text:  ['The', 'introduction', 'of', 'the', 'gas', 'stove', ',', 'the', 'electric', 'mixer', 'and', 'the', 'food', 'processor', 'made', 'the', 'process', 'easier', ',', 'faster', 'and', 'more', 'reliable', '.']
e1 is: 1
e2 is: 16
num:  1
And, of course, everyone wants to prove the truth of their beliefs through experience, but the <e1>belief</e1> begets the <e2>experience</e2>.
q1:  belief
q2:  experience
text:  ['And', ',', 'of', 'course', ',', 'everyone', 'wants', 'to', 'prove', 'the', 'truth', 'of', 'their', 'beliefs', 'through', 'experience', ',', 'but', 'the', 'belief', 'begets', 'the', 'experience', '.']
e1 is: 19
e2 is: 22
num:  2
As for camera/lens info I could care less - the <e1>photographer</e1> made the <e2>picture</e2>, not the camera - although the film used is sometimes good to know.
q1:  

q1:  soil
q2:  reputable
text:  ['Ultimately', ',', 'your', 'garden', 'will', 'do', 'well', 'with', 'either', 'of', 'these', 'soils', 'as', 'long', 'as', 'the', 'soil', 'comes', 'from', 'a', 'reputable', 'dealer', '.']
e1 is: 16
e2 is: 20
num:  22
The <e1>spring molt</e1> produces <e2>mottled brown feathers</e2>, making the female almost invisible as she sits on her nest.
q1:  spring
q2:  mottled
text:  ['The', 'spring', 'molt', 'produces', 'mottled', 'brown', 'feathers', ',', 'making', 'the', 'female', 'almost', 'invisible', 'as', 'she', 'sits', 'on', 'her', 'nest', '.']
e1 is: 1
e2 is: 4
num:  23
I have to say the <e1>soy milk</e1> makes a <e2>great coffee creamer</e2> (and with whatever else you can cook it).
q1:  soy
q2:  great
text:  ['I', 'have', 'to', 'say', 'the', 'soy', 'milk', 'makes', 'a', 'great', 'coffee', 'creamer', '-LRB-', 'and', 'with', 'whatever', 'else', 'you', 'can', 'cook', 'it', '-RRB-', '.']
e1 is: 5
e2 is: 9
num:  24
If the product makes it over that first hurdl

q1:  ballet
q2:  company
text:  ['Nine-year-old', 'Chloe', 'Clark', 'joined', 'a', 'ballet', 'company', 'on', 'stage', 'last', 'Monday', '.']
e1 is: 5
e2 is: 6
num:  42
When I start the engine the <e1>car</e1> makes a <e2>screaching sound</e2>.
q1:  car
q2:  screaching
text:  ['When', 'I', 'start', 'the', 'engine', 'the', 'car', 'makes', 'a', 'screaching', 'sound', '.']
e1 is: 6
e2 is: 9
num:  43
In response to the problem, the <e1>assembly</e1> composed the following <e2>statement</e2>.
q1:  assembly
q2:  statement
text:  ['In', 'response', 'to', 'the', 'problem', ',', 'the', 'assembly', 'composed', 'the', 'following', 'statement', '.']
e1 is: 7
e2 is: 11
num:  44
When man sinned, the occasion of his fall was a tree, and the <e1>ground</e1> cursed brought forth <e2>thorns</e2>.
q1:  ground
q2:  thorns
text:  ['When', 'man', 'sinned', ',', 'the', 'occasion', 'of', 'his', 'fall', 'was', 'a', 'tree', ',', 'and', 'the', 'ground', 'cursed', 'brought', 'forth', 'thorns', '.']
e1 is: 15
e2 i

q1:  documentary
q2:  maker
text:  ['Alex', "'s", 'skills', 'as', 'an', 'editor', 'and', 'a', 'documentary', 'maker', 'allowed', 'us', 'to', 'be', 'able', 'to', 'shoot', 'it', 'and', 'turn', 'it', 'in', 'that', 'short', 'amount', 'of', 'time', '.']
e1 is: 8
e2 is: 9
num:  66
At some point in or after the year 857 the <e1>priest</e1> composed the <e2>book</e2>.
q1:  priest
q2:  book
text:  ['At', 'some', 'point', 'in', 'or', 'after', 'the', 'year', '857', 'the', 'priest', 'composed', 'the', 'book', '.']
e1 is: 10
e2 is: 13
num:  67
In prolific suspense novelist Johansen's energetic new Eve Duncan thriller (Countdown, 2005, etc.), the <e1>forensics sculptor</e1> makes a <e2>dangerous deal</e2> with a Colombian weapons dealer.
q1:  forensics
q2:  dangerous
text:  ['In', 'prolific', 'suspense', 'novelist', 'Johansen', "'s", 'energetic', 'new', 'Eve', 'Duncan', 'thriller', '-LRB-', 'Countdown', ',', '2005', ',', 'etc.', '-RRB-', ',', 'the', 'forensics', 'sculptor', 'makes', 'a', 'dangerous'

q1:  afterglow
q2:  shock
text:  ['According', 'to', 'this', 'model', ',', 'the', 'afterglow', 'is', 'produced', 'by', 'shock', 'waves', 'in', 'the', 'jets', 'of', 'tenuous', 'gas', 'that', 'shoot', 'from', 'the', 'blast', 'at', 'almost', 'the', 'speed', 'of', 'light', '.']
e1 is: 6
e2 is: 10
num:  88
The <e1>revolution</e1> brought forth impassioned <e2>pleas</e2> condemning the common law for not respecting the interests of the people and calling for law reform.
q1:  revolution
q2:  pleas
text:  ['The', 'revolution', 'brought', 'forth', 'impassioned', 'pleas', 'condemning', 'the', 'common', 'law', 'for', 'not', 'respecting', 'the', 'interests', 'of', 'the', 'people', 'and', 'calling', 'for', 'law', 'reform', '.']
e1 is: 1
e2 is: 5
num:  89
The <e1>move</e1> makes a <e2>mockery</e2> of China's long-standing opposition to such capabilities in the hands of other powers.
q1:  move
q2:  mockery
text:  ['The', 'move', 'makes', 'a', 'mockery', 'of', 'China', "'s", 'long-standing', 'oppositi

In [1147]:
with open('test/relation-3/test_3_data', 'wb') as f:
    pickle.dump((sentences, e1, e2), f)
    f.close()

In [1148]:
os.environ['CLASSPATH'] = r'/usr/local/Cellar/stanford-parser/3.9.1/libexec'
dep_parser = StanfordDependencyParser()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.StanforCoreNLPDependencyParser instead.
  


In [1149]:
def lca(tree, index1, index2):
    node = index1
    path1 = []
    path2 = []
    path1.append(index1)
    path2.append(index2)
    while(node != tree.root):
        node = tree.nodes[node['head']]
        path1.append(node)
    node = index2
    while(node != tree.root):
        node = tree.nodes[node['head']]
        path2.append(node)
    for l1, l2 in zip(path1[::-1],path2[::-1]):
        if(l1==l2):
            temp = l1
    return temp


def path_lca(tree, node, lca_node):
    path = []
    path.append(node)
    while(node != lca_node):
        node = tree.nodes[node['head']]
        path.append(node)
    return path

In [1150]:
f = open('test/relation-3/test_3_data', 'rb')
#print(f)
sentences, e1, e2 = pickle.load(f)
print(sentences[:10])
print('e1:', e1[0:10])
print('e2:', e2[:10])

print(len(sentences))

f.close()

['The introduction of the gas stove , the electric mixer and the food processor made the process easier , faster and more reliable .', 'And , of course , everyone wants to prove the truth of their beliefs through experience , but the belief begets the experience .', 'As for camera/lens info I could care less - the photographer made the picture , not the camera - although the film used is sometimes good to know .', 'These first person accounts humanize history as readers share the experiences of American prosecutors , security personnel , journalists , and even the architect who designed the courtroom .', 'We even heard from descendants of Gustav Schoenike who was the violin maker .', 'Outraged , upon his release and return to England , the artist produced the virulently anti-French painting based on his experience there .', 'But not every run of the experiment produced the desired state - a very large number of failed tries accompanied each successful creation .', 'The company develope

In [1151]:
word_path1 = []
word_path2 = []
rel_path1 = []
rel_path2 = []
pos_path1 = []
pos_path2 = []
for i in range(len(sentences)):
    word_path1.append(0)
    word_path2.append(0)
    rel_path1.append(0)
    rel_path2.append(0)
    pos_path1.append(0)
    pos_path2.append(0)


In [1152]:
for i in range(len(sentences)):
    try:
        parse_tree = dep_parser.raw_parse(sentences[i])
        for trees in parse_tree:
            tree = trees
        node1 = tree.nodes[e1[i]+1]
        node2 = tree.nodes[e2[i]+1]
        if node1['address']!=None and node2['address']!=None:
            print(i, "success")
            lca_node = lca(tree, node1, node2)
            print('lca: ', lca_node)
            #print('\n')
            path1 = path_lca(tree, node1, lca_node)
            path2 = path_lca(tree, node2, lca_node)
            print('path1: ', path1)
            print('path2: ', path2)
            #print('\n')
            word_path1[i] = [p["word"] for p in path1]
            word_path2[i] = [p["word"] for p in path2]
            print('word_path1: ', word_path1)
            print('word_path2: ', word_path2)
            #print('\n')
            rel_path1[i] = [p["rel"] for p in path1]
            rel_path2[i] = [p["rel"] for p in path2]
            print('rel_path1: ', rel_path1)
            print('rel_path2: ', rel_path2)
            #print('\n')
            pos_path1[i] = [p["tag"] for p in path1]
            pos_path2[i] = [p["tag"] for p in path2]
            print('pos_path1: ',  pos_path1)
            print('pos_path2: ',  pos_path2)
            print('\n')
        else:
            print(i, node1["address"], node2["address"])
    except AssertionError:
        print(i, "error")
    

0 success
lca:  {'address': 15, 'word': 'made', 'lemma': '_', 'ctag': 'VBD', 'tag': 'VBD', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [2], 'xcomp': [18]}), 'rel': 'root'}
path1:  [{'address': 2, 'word': 'introduction', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 15, 'deps': defaultdict(<class 'list'>, {'det': [1], 'nmod': [6]}), 'rel': 'nsubj'}, {'address': 15, 'word': 'made', 'lemma': '_', 'ctag': 'VBD', 'tag': 'VBD', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [2], 'xcomp': [18]}), 'rel': 'root'}]
path2:  [{'address': 17, 'word': 'process', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 18, 'deps': defaultdict(<class 'list'>, {'det': [16]}), 'rel': 'nsubj'}, {'address': 18, 'word': 'easier', 'lemma': '_', 'ctag': 'JJR', 'tag': 'JJR', 'feats': '_', 'head': 15, 'deps': defaultdict(<class 'list'>, {'nsubj': [17], 'conj': [20, 23], 'cc': [21]}), 'rel': 'xcomp'}, {'address': 15, 'word': 'made', 'lemm

3 success
lca:  {'address': 11, 'word': 'experiences', 'lemma': '_', 'ctag': 'NNS', 'tag': 'NNS', 'feats': '_', 'head': 9, 'deps': defaultdict(<class 'list'>, {'det': [10], 'nmod': [14], 'acl:relcl': [26]}), 'rel': 'dobj'}
path1:  [{'address': 24, 'word': 'architect', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 14, 'deps': defaultdict(<class 'list'>, {'advmod': [22], 'det': [23]}), 'rel': 'conj'}, {'address': 14, 'word': 'prosecutors', 'lemma': '_', 'ctag': 'NNS', 'tag': 'NNS', 'feats': '_', 'head': 11, 'deps': defaultdict(<class 'list'>, {'case': [12], 'amod': [13], 'conj': [17, 19, 24], 'cc': [21]}), 'rel': 'nmod'}, {'address': 11, 'word': 'experiences', 'lemma': '_', 'ctag': 'NNS', 'tag': 'NNS', 'feats': '_', 'head': 9, 'deps': defaultdict(<class 'list'>, {'det': [10], 'nmod': [14], 'acl:relcl': [26]}), 'rel': 'dobj'}]
path2:  [{'address': 28, 'word': 'courtroom', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 26, 'deps': defaultdict(<class 'list'>

6 success
lca:  {'address': 8, 'word': 'produced', 'lemma': '_', 'ctag': 'VBD', 'tag': 'VBD', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'cc': [1], 'nsubj': [4], 'dobj': [11]}), 'rel': 'root'}
path1:  [{'address': 4, 'word': 'run', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 8, 'deps': defaultdict(<class 'list'>, {'neg': [2], 'det': [3], 'nmod': [7]}), 'rel': 'nsubj'}, {'address': 8, 'word': 'produced', 'lemma': '_', 'ctag': 'VBD', 'tag': 'VBD', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'cc': [1], 'nsubj': [4], 'dobj': [11]}), 'rel': 'root'}]
path2:  [{'address': 10, 'word': 'desired', 'lemma': '_', 'ctag': 'VBN', 'tag': 'VBN', 'feats': '_', 'head': 11, 'deps': defaultdict(<class 'list'>, {}), 'rel': 'amod'}, {'address': 11, 'word': 'state', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 8, 'deps': defaultdict(<class 'list'>, {'det': [9], 'amod': [10], 'dep': [16]}), 'rel': 'dobj'}, {'address': 8, 'word': 'produc

9 success
lca:  {'address': 17, 'word': 'wine', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 14, 'deps': defaultdict(<class 'list'>, {'case': [15], 'det': [16], 'acl': [18]}), 'rel': 'nmod'}
path1:  [{'address': 17, 'word': 'wine', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 14, 'deps': defaultdict(<class 'list'>, {'case': [15], 'det': [16], 'acl': [18]}), 'rel': 'nmod'}]
path2:  [{'address': 21, 'word': 'vintners', 'lemma': '_', 'ctag': 'NNS', 'tag': 'NNS', 'feats': '_', 'head': 18, 'deps': defaultdict(<class 'list'>, {'case': [19], 'nmod:poss': [20]}), 'rel': 'nmod'}, {'address': 18, 'word': 'produced', 'lemma': '_', 'ctag': 'VBN', 'tag': 'VBN', 'feats': '_', 'head': 17, 'deps': defaultdict(<class 'list'>, {'nmod': [21]}), 'rel': 'acl'}, {'address': 17, 'word': 'wine', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 14, 'deps': defaultdict(<class 'list'>, {'case': [15], 'det': [16], 'acl': [18]}), 'rel': 'nmod'}]
word_path1:  [['int

12 success
lca:  {'address': 4, 'word': 'hormone', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [1], 'cop': [2], 'det': [3], 'acl': [6]}), 'rel': 'root'}
path1:  [{'address': 1, 'word': 'Insulin', 'lemma': '_', 'ctag': 'NNP', 'tag': 'NNP', 'feats': '_', 'head': 4, 'deps': defaultdict(<class 'list'>, {}), 'rel': 'nsubj'}, {'address': 4, 'word': 'hormone', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [1], 'cop': [2], 'det': [3], 'acl': [6]}), 'rel': 'root'}]
path2:  [{'address': 9, 'word': 'pancreas', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 6, 'deps': defaultdict(<class 'list'>, {'case': [7], 'det': [8]}), 'rel': 'nmod'}, {'address': 6, 'word': 'produced', 'lemma': '_', 'ctag': 'VBN', 'tag': 'VBN', 'feats': '_', 'head': 4, 'deps': defaultdict(<class 'list'>, {'advmod': [5], 'nmod': [9]}), 'rel': 'acl'}, {'address': 4, 'word': 'hormon

...

88 success
lca:  {'address': 3, 'word': 'brought', 'lemma': '_', 'ctag': 'VBD', 'tag': 'VBD', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [2], 'compound:prt': [4], 'dobj': [6]}), 'rel': 'root'}
path1:  [{'address': 2, 'word': 'revolution', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 3, 'deps': defaultdict(<class 'list'>, {'det': [1]}), 'rel': 'nsubj'}, {'address': 3, 'word': 'brought', 'lemma': '_', 'ctag': 'VBD', 'tag': 'VBD', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [2], 'compound:prt': [4], 'dobj': [6]}), 'rel': 'root'}]
path2:  [{'address': 6, 'word': 'pleas', 'lemma': '_', 'ctag': 'NNS', 'tag': 'NNS', 'feats': '_', 'head': 3, 'deps': defaultdict(<class 'list'>, {'amod': [5], 'acl': [7]}), 'rel': 'dobj'}, {'address': 3, 'word': 'brought', 'lemma': '_', 'ctag': 'VBD', 'tag': 'VBD', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [2], 'compound:prt': [4], 'dobj': [6]}), 'rel': 'root'}]


89 success
lca:  {'address': 3, 'word': 'makes', 'lemma': '_', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [2], 'dobj': [5], 'nmod': [13]}), 'rel': 'root'}
path1:  [{'address': 2, 'word': 'move', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 3, 'deps': defaultdict(<class 'list'>, {'det': [1]}), 'rel': 'nsubj'}, {'address': 3, 'word': 'makes', 'lemma': '_', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [2], 'dobj': [5], 'nmod': [13]}), 'rel': 'root'}]
path2:  [{'address': 5, 'word': 'mockery', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 3, 'deps': defaultdict(<class 'list'>, {'det': [4], 'nmod': [10]}), 'rel': 'dobj'}, {'address': 3, 'word': 'makes', 'lemma': '_', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [2], 'dobj': [5], 'nmod': [13]}), 'rel': 'root'}]
word_path1:  [['introduction', '

90 success
lca:  {'address': 8, 'word': 'produces', 'lemma': '_', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [7], 'dobj': [11]}), 'rel': 'root'}
path1:  [{'address': 7, 'word': 'fuser', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 8, 'deps': defaultdict(<class 'list'>, {'nmod:poss': [4], 'amod': [6]}), 'rel': 'nsubj'}, {'address': 8, 'word': 'produces', 'lemma': '_', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [7], 'dobj': [11]}), 'rel': 'root'}]
path2:  [{'address': 11, 'word': 'page', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 8, 'deps': defaultdict(<class 'list'>, {'det': [9], 'amod': [10], 'nmod': [15]}), 'rel': 'dobj'}, {'address': 8, 'word': 'produces', 'lemma': '_', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [7], 'dobj': [11]}), 'rel': 'root'}]
word_path1:  [['introduction',

91 success
lca:  {'address': 4, 'word': 'produced', 'lemma': '_', 'ctag': 'VBD', 'tag': 'VBD', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [3], 'dobj': [7]}), 'rel': 'root'}
path1:  [{'address': 3, 'word': 'community', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 4, 'deps': defaultdict(<class 'list'>, {'det': [1], 'amod': [2]}), 'rel': 'nsubj'}, {'address': 4, 'word': 'produced', 'lemma': '_', 'ctag': 'VBD', 'tag': 'VBD', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [3], 'dobj': [7]}), 'rel': 'root'}]
path2:  [{'address': 7, 'word': 'personality', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 4, 'deps': defaultdict(<class 'list'>, {'det': [5], 'amod': [6], 'cc': [9], 'conj': [11]}), 'rel': 'dobj'}, {'address': 4, 'word': 'produced', 'lemma': '_', 'ctag': 'VBD', 'tag': 'VBD', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [3], 'dobj': [7]}), 'rel': 'root'}]
word_path1:  [['int

92 success
lca:  {'address': 4, 'word': 'produces', 'lemma': '_', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [3], 'dobj': [5]}), 'rel': 'root'}
path1:  [{'address': 3, 'word': 'rule', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 4, 'deps': defaultdict(<class 'list'>, {'det': [1], 'amod': [2]}), 'rel': 'nsubj'}, {'address': 4, 'word': 'produces', 'lemma': '_', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [3], 'dobj': [5]}), 'rel': 'root'}]
path2:  [{'address': 5, 'word': 'sentences', 'lemma': '_', 'ctag': 'NNS', 'tag': 'NNS', 'feats': '_', 'head': 4, 'deps': defaultdict(<class 'list'>, {'nmod': [8]}), 'rel': 'dobj'}, {'address': 4, 'word': 'produces', 'lemma': '_', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [3], 'dobj': [5]}), 'rel': 'root'}]
word_path1:  [['introduction', 'made'], ['belief', 'begets'],

In [1153]:
file = open('test/relation-3/test_3_paths', 'wb')
pickle.dump([word_path1, word_path2, rel_path1, rel_path2, pos_path1, pos_path2], file)

l = len(word_path1)
print(l)


93


In [1154]:
f = open('test' + '/relation-3/test_3_paths', 'rb')
word_p1, word_p2, dep_p1, dep_p2, pos_p1, pos_p2 = pickle.load(f)
print(len(dep_p1))
#print('word_p1:', word_p1)
f.close()
print(word_p1[0])
    
length = len(word_p1)
print(length)

num_batches = int(length/batch_size)

print(word_p1)

for i in range(length):
    for j, word in enumerate(word_p1[i]):
        word = word.lower()
        word_p1[i][j] = word if word in word2id else unknown_token 
    for k, word in enumerate(word_p2[i]):
        word = word.lower()
        word_p2[i][k] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p1[i]):
        dep_p1[i][l] = d if d in dep2id else 'OTH'
    for m, d in enumerate(dep_p2[i]):
        dep_p2[i][m] = d if d in dep2id else 'OTH'

word_p1_ids = np.ones([length, max_len_path],dtype=int)
word_p2_ids = np.ones([length, max_len_path],dtype=int)
pos_p1_ids = np.ones([length, max_len_path],dtype=int)
pos_p2_ids = np.ones([length, max_len_path],dtype=int)
dep_p1_ids = np.ones([length, max_len_path],dtype=int)
dep_p2_ids = np.ones([length, max_len_path],dtype=int)

rel_ids = np.array([rel2id[rel[0]] for rel in relations])
print(len(rel_ids))

path1_len = np.array([len(w) for w in word_p1], dtype=int)
path2_len = np.array([len(w) for w in word_p2])

for i in range(length):
    for j, w in enumerate(word_p1[i]):
        word_p1_ids[i][j] = word2id[w]
    for j, w in enumerate(word_p2[i]):
        word_p2_ids[i][j] = word2id[w]
    for j, w in enumerate(pos_p1[i]):
        pos_p1_ids[i][j] = pos_tag(w)
    for j, w in enumerate(pos_p2[i]):
        pos_p2_ids[i][j] = pos_tag(w)
    for j, w in enumerate(dep_p1[i]):
        dep_p1_ids[i][j] = dep2id[w]
    for j, w in enumerate(dep_p2[i]):
        dep_p2_ids[i][j] = dep2id[w]


93
['introduction', 'made']
93
[['introduction', 'made'], ['belief', 'begets'], ['photographer'], ['architect', 'prosecutors', 'experiences'], ['violin', 'maker'], ['artist', 'produced'], ['run', 'produced'], ['company', 'developed'], ['regiment', 'composed'], ['wine'], ['People', 'harvest'], ['wine'], ['Insulin', 'hormone'], ['inversion', 'produced'], ['organ'], ['programmer', 'makes'], ['prescriber', 'makes'], ['brothers', 'created'], ['sub-catchment', 'farmed'], ['bus', 'company'], ['site', 'dry'], ['soil', 'comes'], ['spring', 'molt', 'produces'], ['soy', 'milk', 'makes'], ['shopper', 'makes'], ['story'], ['conflict'], ['institution', 'provide'], ['generator', 'produced'], ['farmers', 'market'], ['mass', 'cultivated'], ['filament', 'emits'], ['tick', 'produced'], ['minicab', 'company'], ['biologists', 'created'], ['aristocracy', 'composed'], ['producer'], ['draft', 'makes'], ['sculptor', 'produced'], ['heroin', 'producer'], ['toolkits', 'provide'], ['ballet', 'company'], ['car', 'm

In [1155]:
# training accuracy
relation = ['Product-Producer(e1,e2)', 'Product-Producer(e2,e1)', 'others']
all_predictions = []

for j in range(num_batches):
    path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
    word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
    pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
    dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]


    feed_dict = {
        path_length:path_dict,
        word_ids:word_dict,
        pos_ids:pos_dict,
        dep_ids:dep_dict,
        }
    
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)
    
y_pred = []
y_pred_rel = []

for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(int(pred))
        y_pred_rel.append(relation[pred])



#print("training accuracy", accuracy, '\n')
print('the predicted relation id is: ', y_pred)
print(len(y_pred))

print('the predicted relation is: ', y_pred_rel , '\n')
print(len(y_pred_rel))





f=open('test/relation-3/test_3_pred.txt', 'w')
for i in y_pred_rel:
    f.write(i + '\n')

the predicted relation id is:  [1, 2, 1, 2, 0, 1, 2, 1, 2, 0, 1, 0, 0, 0, 2, 1, 1, 2, 2, 0, 1, 0, 1, 2, 1, 0, 2, 2, 0, 0, 1, 1, 0, 0, 0, 2, 0, 1, 2, 0, 2, 0, 1, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 1, 2, 0, 1, 1, 0, 1, 1, 2, 2, 2, 0, 2, 0, 1, 2, 2, 2, 0, 2, 1, 1, 2, 1]
93
the predicted relation is:  ['Product-Producer(e2,e1)', 'others', 'Product-Producer(e2,e1)', 'others', 'Product-Producer(e1,e2)', 'Product-Producer(e2,e1)', 'others', 'Product-Producer(e2,e1)', 'others', 'Product-Producer(e1,e2)', 'Product-Producer(e2,e1)', 'Product-Producer(e1,e2)', 'Product-Producer(e1,e2)', 'Product-Producer(e1,e2)', 'others', 'Product-Producer(e2,e1)', 'Product-Producer(e2,e1)', 'others', 'others', 'Product-Producer(e1,e2)', 'Product-Producer(e2,e1)', 'Product-Producer(e1,e2)', 'Product-Producer(e2,e1)', 'others', 'Product-Producer(e2,e1)', 'Product-Producer(e1,e2)', 'others', 'others', 'Product-Producer(e1,e2)', 'Product-Producer(e1,e2)', 'Product-Producer

In [1156]:
with open('test/relation-3-test.txt', encoding="latin") as f:
    file = f.read()
    print(file)

141 "The <e1>introduction</e1> of the gas stove, the electric mixer and the food processor made the <e2>process</e2> easier, faster and more reliable."
WordNet(e1) = "introduction%1:04:02::", WordNet(e2) = "process%1:04:00::", Product-Producer(e2, e1) = "?", Query = "the * made the *"

142 "And, of course, everyone wants to prove the truth of their beliefs through experience, but the <e1>belief</e1> begets the <e2>experience</e2>."
WordNet(e1) = "belief%1:09:00::", WordNet(e2) = "experience%1:09:00::", Product-Producer(e2, e1) = "?", Query = "the * begets the *"

143 "As for camera/lens info I could care less - the <e1>photographer</e1> made the <e2>picture</e2>, not the camera - although the film used is sometimes good to know."
WordNet(e1) = "photographer%1:18:00::", WordNet(e2) = "picture%1:06:02::", Product-Producer(e2, e1) = "?", Query = "the * made the *"

144 "These first person accounts humanize history as readers share the experiences of American prosecutors, security personne

In [1157]:
lines, rels, rel_test = [], [], []
for line in open("test/relation-3-test.txt",  encoding="latin"):
    lines.append(line.strip())
#print('lines is: ', lines)
        
d=0        
for i, w in enumerate(lines):
    m = re.match(r'^([0-9]+)\s"(.+)"$', w.strip())
    #print(m)
    if (m is not None):
        #print(m.group(0))
        indx = lines.index(m.group(0))
        #print(d)
        d+=1
        #print(str(lines[indx+1].split(',')[2].strip()) + ',' + str(lines[indx+1].split(',')[3].strip()))
        #rels.append(str(lines[indx+1].split(',')[2].strip()) + ',' + str(lines[indx+1].split(',')[3].strip()))
        rels.append(str(lines[indx+1].split(',')[2].strip()) + ',' + str(lines[indx+1].split(',')[3].strip()))

for i in rels:
    rel_test.append(i.split(' =')[0])
print(len(rel_test))

93


In [1158]:
f = open('test/relation-3/pred.txt', 'w+') 
         
for i in range(len(y_pred)):
    if rel_test[i] == y_pred_rel[i].strip(' '):
        f.write('True' + '\n')
    else:
        f.write('False' + '\n')
        